In [1]:
import tasks
from utils import parse_config

configuration = parse_config("config")

raw_data = configuration["raw_data"]
TABLES = configuration["TABLES"]
SCD_types = configuration["SCD"]

db_master = configuration["db_master"]

db_rel = configuration["db_rel"]
db_dim = configuration["db_dim"]

schema_rel = configuration["schema_rel"]
schema_dim = configuration["schema_dim"]

In [2]:
conn_master = tasks.connect_to(db_master)

tasks.drop_database_if_exists(conn_master)
tasks.create_database(conn_master)

conn_master.close()

Connection string: DRIVER=/opt/homebrew/lib/libmsodbcsql.18.dylib;SERVER=localhost,1433;DATABASE=master;TrustServerCertificate=yes;UID=sa;PWD=bigStrong1234Password!re
Connected    

Dropped databases
Created databases


In [3]:
conn_ER = tasks.connect_to(db_rel)

tasks.create_database(conn_ER)

tasks.drop_constraints_if_exist(conn_ER, db_rel, schema_rel)

for table in TABLES:
    tasks.drop_table_if_exists(conn_ER, table, db_rel, schema_rel)
    tasks.create_table(conn_ER, table, db_rel, schema_rel)
    tasks.insert_into_table(conn_ER, raw_data, table, db_rel, schema_rel)

tasks.set_constraints(conn_ER, db_rel, schema_rel)

conn_ER.close()

Connection string: DRIVER=/opt/homebrew/lib/libmsodbcsql.18.dylib;SERVER=localhost,1433;DATABASE=Orders_RELATIONAL_DB;TrustServerCertificate=yes;UID=sa;PWD=bigStrong1234Password!re
Connected    

Created databases
The dbo.Categories table from the database Orders_RELATIONAL_DB has been dropped
The dbo.Categories table from the database Orders_RELATIONAL_DB has been created
Inserting into Categories...
8 rows have been inserted into the Orders_RELATIONAL_DB.dbo.Categories table

The dbo.Customers table from the database Orders_RELATIONAL_DB has been dropped
The dbo.Customers table from the database Orders_RELATIONAL_DB has been created
Inserting into Customers...
91 rows have been inserted into the Orders_RELATIONAL_DB.dbo.Customers table

The dbo.Employees table from the database Orders_RELATIONAL_DB has been dropped
The dbo.Employees table from the database Orders_RELATIONAL_DB has been created
Inserting into Employees...
9 rows have been inserted into the Orders_RELATIONAL_DB.dbo.Emp

In [4]:
conn_DW = tasks.connect_to(db_dim)

for table, SCD_type in SCD_types.items():
    
    tasks.drop_dim_table_if_exists(conn_DW, table, SCD_type, db_dim, schema_dim)
    
    tasks.create_dim_table(conn_DW, table, SCD_type, db_dim, schema_dim)
    
    tasks.update_dim_table(conn_DW, table, SCD_type, db_dim, db_rel, schema_dim, schema_rel)


conn_DW.close()

Connection string: DRIVER=/opt/homebrew/lib/libmsodbcsql.18.dylib;SERVER=localhost,1433;DATABASE=Orders_DIMENSIONAL_DW;TrustServerCertificate=yes;UID=sa;PWD=bigStrong1234Password!re
Connected    

The dbo.dim_Categories_SCD3 table from the database Orders_DIMENSIONAL_DW has been dropped
The dbo.dim_Categories_SCD3 table from the database Orders_DIMENSIONAL_DW has been created

Executing Categories_SCD3_ETL...
Updated dim table Categories_SCD3

The dbo.dim_Customers_SCD3 table from the database Orders_DIMENSIONAL_DW has been dropped
The dbo.dim_Customers_SCD3 table from the database Orders_DIMENSIONAL_DW has been created

Executing Customers_SCD3_ETL...
Updated dim table Customers_SCD3

The dbo.dim_Employees_SCD2 table from the database Orders_DIMENSIONAL_DW has been dropped
The dbo.dim_Employees_SCD2 table from the database Orders_DIMENSIONAL_DW has been created

Executing Employees_SCD2_ETL...
Updated dim table Employees_SCD2

The dbo.dim_Products_SCD4 table from the database Orders_D